# Case Study: Truss Rectangular

In [ ]:
# OS functionalities (to deal with files)
import os

# Enhanced Iteration capabilities (to use cross-products)
import itertools

# Data processing packages
import pandas as pd
import numpy as np

# Global Setup for Processing Data

In this section, we define all the global constants that will be used thourought the script. By defining them up here, we are making this notebook purely parametric and reusable for other projects (as long as the folder structure remains the same).  

In [ ]:
# Folders Structure
base_folder = os.getcwd()
results_folder = os.path.join(base_folder, "algorithms")
indicators_folder = os.path.join(base_folder, "performance_indicators")
output_folder = "./output/"

# CSV File Configuration
has_header=True
files_sep= ","
file_extension = 'csv'

# Optimization Settings
runs = [1, 2, 3]
nruns = len(runs)
max_evals = 600

## Problem Definition (in the files) 
### Variables
Material = 3
Bar_Radius = 4
N_Bars = 5
Max_Pyramid_Height = 6

vars_cols = [Material, Bar_Radius, N_Bars, Max_Pyramid_Height]

### Objectives  
Max_Displacement = 7
Cost = 8

objs_cols = [Max_Displacement, Cost]

Time = 0
relevant_cols = [Time] + vars_cols + objs_cols 

## Multi-Objective Optimization Algorithms
### Metaheuristics
pop_size = 30
metaheuristics = ["SMPSO", "OMOPSO", "SPEA2", "NSGAII", "MOEAD", "PESA2"]

### Model Based (or metamodel)
metamodels_base = ["RFR", "ETR", "GPR"] 
metamodels_strategies = ["SPEA2"]
metamodels_algorithms = [f"{b}_{s}" for (b, s) in itertools.product(metamodels_base, metamodels_strategies)]

# all_algorithms = metaheuristics
all_algorithms = metaheuristics + metamodels_algorithms
n_algorithms = len(all_algorithms)

### Filenames with the results 
filenames = [f"{a}_results_0{r}.{file_extension}" for (r, a) in itertools.product(runs, all_algorithms)]
filenames

In [ ]:
# Sanity check: verify the file names are what we expected
filenames[::3]

## Input/Output (IO) Methods 

In this subsection we create the methods that will be responsible for loading the data from the files. To manipulate the data, we will use pandas.DataFrame data structure. This can be easily manipulated and different statistics can be computed on top of these abstractions. 

In [ ]:
def load_results(filenames, base_folder=results_folder, 
                 has_header=True, keep_header=False, 
                 sep=files_sep, usecols=relevant_cols,  
                 max_lines=max_evals):
    """Loads the data from the specified `base_folder` using the `filenames`.
    It assumes the filenames
    """
    read_args = { 
        "header": 'infer' if keep_header else None, 
        "sep": sep,
        "usecols": usecols if usecols else None,
        "skiprows": 1 if has_header and not keep_header else 0,
    }
    filepaths = [os.path.join(base_folder, f) for f in filenames]        
    if max_lines:
        return [pd.read_csv(f, **read_args)[0:max_lines] for f in filepaths]
    else:
        return [pd.read_csv(f, **read_args) for f in filepaths]

In [ ]:
# Always confirm whether the results are according to what you expected
examples = load_results(filenames, max_lines=max_evals)
examples[0].head()

In [ ]:
list(filter(lambda x: x > max_evals, [len(e) for e in examples]))

In [ ]:
def get_run_indices(dfs, run, n_algorithms=n_algorithms):
    """Returns the dataframes that correspond to the specified `run`. 
    This assumes that the dataframes are read per run, that means that 
    if we run two algorithms (SMPSO and SPEA2) for 3 runs each, this 
    method assumes that it was read in the following order:
    
    > SMPSO_run1
    > SPEA2_run1
    > SMPSO_run2
    > SPEA2_run2
    > SMPSO_run3
    > SPEA2_run3
    
    In that case, this invocation `get_run_indices(dfs, 1, n_algorithms=2)`
    will return: 
        dfs[0:2] 
    """
    run -= 1
    return dfs[run*n_algorithms:(run+1)*n_algorithms]

## Pareto Dominance Methods
This section contains methods related to the Pareto optimality (or [Pareto Efficiency](https://en.wikipedia.org/wiki/Pareto_efficiency)).

In [ ]:
# **IMPORTANT NOTE**: This function assumes that your problem is a minimization problem for every objective dimension.
def weakly_dominates(v0, v1):
    """Computes whether v0 dominates v1, i.e., whether at least one objective
    is better (in this case, smaller) than some other)
    """
    return np.all(v0 <= v1) and np.any(v0 < v1)
    
# Sanity Check (:
print("(Expected) True   (Obtained)", weakly_dominates(np.array([1, 1]), np.array([2, 2])))
print("(Expected) True   (Obtained)", weakly_dominates(np.array([2, 1]), np.array([2, 2])))
print("(Expected) False  (Obtained)", weakly_dominates(np.array([2, 2]), np.array([1, 1]))) 
print("(Expected) True   (Obtained)", weakly_dominates(np.array([1, 2]), np.array([2, 2]))) 
print("(Expected) False  (Obtained)", weakly_dominates(np.array([1, 3]), np.array([3, 1])))
print("(Expected) False  (Obtained)", weakly_dominates(np.array([3, 1]), np.array([1, 3]))) 
print("(Expected) False  (Obtained)", weakly_dominates(np.array([1, 1]), np.array([1, 1])))  

In [ ]:
def get_non_dominated(V, dominance=weakly_dominates):
    """Computes the optimal and non-optimal solutions. 
    Optimal solutions are called non-dominated and non-optimal 
    solutions are called denominated."""
    nsols, nobjs = V.shape
    
    dominated = np.zeros((nsols, 1))
    dominated_by = np.zeros((nsols, 1))
    for i in range(nsols):
        for j in range(nsols):
            if i != j:
                if dominance(V[j], V[i]):
                    dominated[i] = 1
                    dominated_by[i] = j 
                    break
                    
    return dominated, dominated_by

In [ ]:
def add_isdominated_cols(d, cols=objs_cols):
    """Adds to the provided dataframe columns for Pareto optimal solution."""
    df_copy = d.copy()
    A = np.array(df_copy[cols])
    B, C = get_non_dominated(A)
    df_copy["isDominated"] = pd.DataFrame(B, columns=["isDominated"])
    df_copy["dominatedBy"] = pd.DataFrame(C, columns=["dominatedBy"])
    print(df_copy["isDominated"].value_counts())
    return df_copy

In [ ]:
def get_combined_PF(dfs, drop_cols=relevant_cols, objs_cols=objs_cols):
    """Computes the combined Pareto front based on a set of input dataframes"""
    all_data = pd.concat(dfs)
    if drop_cols:
        all_data = all_data.drop_duplicates(drop_cols)
    all_data = all_data.reset_index()
    return add_isdominated_cols(all_data, cols=objs_cols)

## General Methods
This section contains general purpose methods that can be used in your scripts.

In [ ]:
def broadcasting_multi(df, cols, value):
    cols = cols if isinstance(cols, (list, tuple)) else [cols]
    copy_df = df.copy()
    for col in cols:
        copy_df[col] = df[col] * value
    return copy_df

In [ ]:
def broadcasting_div(df, cols, value):
    cols = cols if isinstance(cols, (list, tuple)) else [cols]
    copy_df = df.copy()
    for col in cols:
        copy_df[col] = df[col] / value
    return copy_df

In [ ]:
def get_symmetric(df, cols):
    """Computes the symmetric value of the provided `cols` and returns a 
    copy of the original dataframe where the values of the specified `cols`
    are symmetric."""
    return broadcasting_multi(df, cols, -1)

In [ ]:
def unscale(df, cols, minimum, step):
    """Computes the real value used for the variables during a discrete optimization process."""
    cols = cols if isinstance(cols, (list, tuple)) else [cols]
    copy_df = df.copy()
    for col in cols:
        copy_df[col] = minimum + step * df[col]
    return copy_df

In [ ]:
def drop_by_value(dfs, col, value):
    """Drops the solutions from the provided dataframes based on a column and a value, 
    changing them inplace."""
    for df in dfs:      
        unfeasible_sols = df.loc[df[col]==value]
        df.drop(unfeasible_sols.index, inplace=True)
        df.index = range(len(df))
    return dfs

In [ ]:
def drop_unfeasibles(dfs):
    """Drops the unfeasible solutions from the provided dataframes, 
    changing them inplace."""
    return drop_by_value(dfs, feasible_col, False)

In [ ]:
def unfeasible_ratio_iter(pop_size=pop_size):
    """Computes the unfeasible ratio per `pop_size`."""
    results = []
    for i in range(0, len(df), pop_size):
        results.append(df[df[i:i + pop_size] == 'false']['feasible'].count() / pop_size)
    iteration = pd.Series(data = range(1, int(len(df) / pop_size) + 1))
    ratio = pd.Series(data = results)
    ratio_iter = {'Iteration': iteration, 'Ratio Unfeasible': ratio} 
    return pd.DataFrame(ratio_iter)

In [ ]:
def get_last_value_iter(df, col_name='hypervolume', iter_size=pop_size):
    """Iterates the values in the `df` in slices of `iter_size` and 
    gets the last value.
    """
    return df[iter_size-1:len(df):iter_size][col_name]

In [ ]:
def indicators_results(results_runs):
    """Computes the statistics mean and standard deviation for the specified runs. 
    
    It assumes the `results_run` are organized as follows: 
        results_runs = [indicators_run1, indicators_run2, indicators_run3], 
    where: 
        indicators_run{i} = [indicators_alg_1_run_{i}, ..., indicators_alg_m_run_{i}]
    """
    average_res = [] 
    std_res = [] 
    
    for algorithm_i in range(n_algorithms):
        # Get the same algorithm from all runs (assuming they were collected in the same way)
        algorithm_results = [r[algorithm_i] for r in results_runs] 
        average_res += [pd.concat(algorithm_results, axis=1).mean(axis=1)]
        std_res += [pd.concat(algorithm_results, axis=1).std(axis=1)]
        
    return average_res, std_res

## Data Visualization

In this section, we visualize the data with visual means. 

In [ ]:
# Visualization Framework
import plotly
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
%matplotlib inline
init_notebook_mode(connected=True)

try:
    import plotly.plotly as py
    plotly.tools.set_credentials_file(username='username', api_key='api_key')
except: 
    import chart_studio
    import chart_studio.plotly as py
    chart_studio.tools.set_credentials_file(username='username', api_key='api_key')
    
# Print plotly's version
plotly.__version__

Let us create the functions that create the graphs

In [ ]:
def get_colors(n, colorscale="viridis"): 
    colors = plt.get_cmap(colorscale).colors
    colors_idx = np.linspace(0, len(colors)-1, n, dtype=int)
    
    colors = [colors[idx] for idx in colors_idx]
    colors_str = [f"rgb({r}, {g}, {b})" for (r, g, b) in colors]
    return colors_str

In [ ]:
def scatter(data, x=None, y=None, names=all_algorithms, 
            colorscale="viridis", colors=None, 
            mode="markers", marker_size=5.5, ln_width=1.5, 
            layout=None):
    
    def get_x_y(d): 
        if isinstance(d, pd.Series):
            return np.array(d.index) + 1, d.values
        elif isinstance(d, pd.DataFrame):
            return d[x], d[y]
        else: 
            return np.arange(len(d)), d      
        
    # ----------------------------------------------
    # Normalize input data
    # ----------------------------------------------
    data = data if isinstance(data, (list, tuple)) else [data]
    colors = get_colors(len(data), colorscale) if colorscale else colors
    
    # ----------------------------------------------
    # Determine the data types of provided inputs
    # ----------------------------------------------
    x_data = []
    y_data = []
    for d in data:
        xx, yy = get_x_y(d)
        x_data += [xx]
        y_data += [yy]

    
    traces = []
    for (i, (x,y)) in enumerate(zip(x_data, y_data)):
        traces += [
            go.Scatter(
                x = x,
                y = y,
                mode = mode,
                name = names[i],
                marker = dict(
                    # Markers size
                    size = marker_size,
                    color=colors[i],
                ),
                line=dict(
                    width=ln_width,
                    color=colors[i]
            )
            )]
    
    kwargs = {} if not layout else {"layout": layout}
    fig = go.Figure(data=traces, **kwargs)
    return py.iplot(fig, filename='simple_scatter')

### Default Layout

In [ ]:
# Default layout for the pareto fronts graphs
layout = go.Layout(
    template="plotly_white",
    autosize=True,
    legend=dict(
        orientation='h'
    ),
    # Define axis
    xaxis=dict(
        autorange=True,
        showgrid=True,
        zeroline=False,
        showline=True,
        ticks='',
        showticklabels=True,
        tickformat='.'
    ),
    yaxis=dict(
        autorange=True,        
        showgrid=True,
        zeroline=False,
        showline=True,
        ticks='',
        showticklabels=True,
        tickformat='.'
    )
)

### Pareto Front

This section contains different functions that explore dataframes having information about the non-dominated and the dominated solutions.

In [ ]:
def create_pf(pf, name, x, y, nd_color='rgb(0,0,255)', ln_width=1.5, marker_size=5.5, d_color=None): 
    traces = []
    
    # Create the non dominated trace (in a different color, as specified in *nd_color*)
    x_pf = pf[pf['isDominated'] == 0][x] 
    y_pf = pf[pf['isDominated'] == 0][y]
    x_pf, y_pf = zip(*sorted(zip(x_pf, y_pf)))
    
    # Get information about the objectives and variables - relevant for an iteractive PF
    info = []
    info.extend(vars_cols)
    info.extend(objs_cols)
    info_pf = np.array(pf[pf['isDominated'] == 0][info])
    info_npf = np.array(pf[pf['isDominated'] == 1][info])
    
    traces += [
        go.Scatter(
            x = x_pf,
            y = y_pf,
            mode = 'lines+markers',
            name = name + " NonDominated",
            customdata = info_pf,
            opacity=1,

            # Layout do marker
            marker=dict(
                color=nd_color,
                size=marker_size
            ),
            line=dict(
                color=nd_color,
                width=ln_width
            )
        )]
    
    if d_color:
        x_npf = pf[pf['isDominated'] == 1][x]
        y_npf = pf[pf['isDominated'] == 1][y]
        
        # Create the dominated trace (in a different color, as specified in *d_color*)
        traces +=[
            go.Scatter(
                x = x_npf,
                y = y_npf,
                mode = 'markers',
                name = name + " Dominated",
                customdata = info_npf,
                opacity=0.55,

                # Layout do Marker
                marker=dict(
                    #color = d_color,
                    color = nd_color,
                    size = 0.55 * marker_size,
                )
            )]

    return traces

In [ ]:
def get_traces(pfs, x, y, draw_dominated=True,
               names=all_algorithms, colorscale='viridis', colors=None,
               tpf=None, tpf_name="Combined_PF", tpf_color='rgb(0,0,0)', 
               layout=layout):
    
    pfs = pfs if isinstance(pfs, (list, tuple)) else [pfs]
    names = names if isinstance(names, (list, tuple)) else [names]
    n_pfs = len(pfs)
    colors = get_colors(n_pfs, colorscale) if colorscale else colors
    
    traces = []
    
    if tpf is not None:
        traces += create_pf(pf=tpf, name=tpf_name, x=x, y=y, ln_width=4, marker_size=10, nd_color=tpf_color)
    
    for (i, pf) in enumerate(pfs):
        d_color = colors[i] if draw_dominated else None
        traces += create_pf(pf=pf, name=names[i], x=x, y=y, nd_color=colors[i], d_color=d_color)

    fig = go.Figure(data=traces, layout=layout)
    return traces

In [ ]:
def create_pfs(pfs, x, y, draw_dominated=True,
               names=all_algorithms, colorscale='viridis', colors=None,
               tpf=None, tpf_name="Combined_PF", tpf_color='rgb(0,0,0)', 
               layout=layout):
    
    traces = get_traces(pfs, x, y, draw_dominated, names, colorscale, colors, tpf, tpf_name, tpf_color, layout)
    
    fig = go.Figure(data=traces, layout=layout)
    return py.iplot(fig, filename='algorithms_pfs_per_run')

---

# Analysis


## Pareto Front Layout

In [ ]:
layout_truss = go.Layout(
    template="plotly_white",
    autosize=True,
    # Legend Position
    legend=dict(
        orientation='h',
        x=-0.01,
        y=-0.2
    ),
    
    # Define axis
    xaxis=dict(
        type="log",
        title="Cost [€]",
        autorange=True, 
        showgrid=True,
        zeroline=False,
        showline=True,
    ),
    yaxis=dict(
        type="log",
        title="Maximum Displacement [m]",
        autorange=True, 
        showgrid=True,
        zeroline=False,
        showline=True,
    )
)

## Read Algorithms

In [ ]:
# Read algorithms  
dfs = load_results(filenames)

In [ ]:
# Sanity check!!
dfs[0].head()

In [ ]:
# Compute non_dominated_solutions (per run)
pfs = [add_isdominated_cols(df) for df in dfs]

In [ ]:
# Sanity check!!
pfs[0].head()

In [ ]:
# Computes combined Pareto Front (optimal solutions found from all the algorithms, all the runs)
combined_pf = get_combined_PF(dfs, drop_cols=relevant_cols)

## Plot for all obtained solution (1 single plot for all algorithms, all runs) 

In [ ]:
create_pfs(pfs, y=Max_Displacement, x=Cost, tpf=combined_pf, names=filenames, 
           colorscale='plasma', draw_dominated=False, layout=layout_truss)

In [ ]:
create_pfs(pfs, y=Max_Displacement, x=Cost, tpf=combined_pf, names=filenames,
           colorscale='plasma', draw_dominated=True, layout=layout_truss)

## Plot algorithms per run (3 plot one for each run) 

In [ ]:
# Get Run 1 
dfs_run_1 = get_run_indices(dfs, 1)
pfs_run_1 = [add_isdominated_cols(df) for df in dfs_run_1]
combined_PF_run_1 = get_combined_PF(pfs_run_1)

In [ ]:
# Get Run 2 
dfs_run_2 = get_run_indices(dfs, 2)
pfs_run_2 = [add_isdominated_cols(df) for df in dfs_run_2]
combined_PF_run_2 = get_combined_PF(pfs_run_2)

In [ ]:
# Get Run 3 
dfs_run_3 = get_run_indices(dfs, 3)
pfs_run_3 = [add_isdominated_cols(df) for df in dfs_run_3]
combined_PF_run_3 = get_combined_PF(pfs_run_3)

In [ ]:
# Get All Runs
# Before running this run the previous cells (the ones that define dfs_run_{i})
pfs_all_runs = []
for i in range(n_algorithms):
    pf=get_combined_PF([dfs_run_1[i], dfs_run_2[i], dfs_run_3[i]])
    pfs_all_runs.append(pf)

### Run 1

In [ ]:
# Create plot with combined pareto front retrieved only from all algorithms of run 1
create_pfs(pfs_run_1, y=Max_Displacement, x=Cost, tpf=combined_PF_run_1,
           names=all_algorithms, draw_dominated=True, layout=layout_truss)

In [ ]:
# Create plot with combined pareto front retrieved from all runs
create_pfs(pfs_run_1, y=Max_Displacement, x=Cost, tpf=combined_pf, 
           names=all_algorithms, draw_dominated=True, layout=layout_truss)

### Run 2

In [ ]:
# Create plot with combined pareto front retrieved only from all algorithms of run 2
create_pfs(pfs_run_2, y=Max_Displacement, x=Cost, tpf=combined_PF_run_2, 
           names=all_algorithms, draw_dominated=True, layout=layout_truss)

In [ ]:
# Create plot with combined pareto front retrieved from all runs
create_pfs(pfs_run_2, x=material_cost, y=sUDI, tpf=combined_pf, 
           names=all_algorithms, draw_dominated=True, layout=layout_truss)

### Run 3

In [ ]:
# Create plot with combined pareto front retrieved only from all algorithms of run 3
create_pfs(pfs_run_3, y=Max_Displacement, x=Cost, tpf=combined_PF_run_3, 
           names=all_algorithms, draw_dominated=True, layout=layout_truss)

In [ ]:
# Create plot with combined pareto front retrieved from all runs
create_pfs(pfs_run_3, x=material_cost, y=sUDI, tpf=combined_pf, 
           names=all_algorithms, draw_dominated=True, layout=layout_truss)

### All Runs

In [ ]:
# Create plot with one PF per algorthim, using the information of the 3 runs
create_pfs(pfs_all_runs, y=Max_Displacement, x=Cost, tpf=None, names=all_algorithms, 
           colorscale=None,
           colors=['#93e0ed', '#307382', #PSOs colors
                   '#2fcce0', '#042780', '#fcb447', '#fcb447', # EAs colors
                   '#db3f30', '#8a39db', '#db30b9'], # MBs colors
           draw_dominated=False, layout=layout_truss)

In [ ]:
# Create plot with one PF per algorthim, using the information of the 3 runs and with the combined PF
create_pfs(pfs_all_runs, y=Max_Displacement, x=Cost, tpf=combined_pf, tpf_color='#3b3b3b',
           names=all_algorithms,
           colorscale=None,
           colors=['#FAA275', '#FF8C61', #PSOs colors
                   '#e77b73', '#ce6a85', '#985277', '#5c374c', # EAs colors
                   '#218380', '#73d2de', '#a0e0e8'], # MBs colors
           draw_dominated=False, layout=layout_truss)

## Plot the Performance Indicators per iteration
After running the optimization processes for the 10 algorithms for 3 runs, we computed their __Hypervolume (HV)__ \[1\], as well as other indicators, such as the __Overall Non-dominated Vector Generated (ONVG)__ \[2\] and the __Set Spacing (SS)__ \[3\].

In [ ]:
idfs = load_results(filenames, base_folder=indicators_folder, has_header=True, keep_header=True, usecols=None)

In [ ]:
idfs[0].head() # Sanity Check

In [ ]:
idfs_run_1 = get_run_indices(idfs, 1)
idfs_run_2 = get_run_indices(idfs, 2)
idfs_run_3 = get_run_indices(idfs, 3)

In [ ]:
hypervolume_run_1 = [get_last_value_iter(df, col_name='hypervolume') for df in idfs_run_1]
hypervolume_run_2 = [get_last_value_iter(df, col_name='hypervolume') for df in idfs_run_2]
hypervolume_run_3 = [get_last_value_iter(df, col_name='hypervolume') for df in idfs_run_3]
hypervolume_per_run = [hypervolume_run_1, hypervolume_run_2, hypervolume_run_3]

In [ ]:
onvg_run_1 = [get_last_value_iter(df, col_name='onvg') for df in idfs_run_1]
onvg_run_2 = [get_last_value_iter(df, col_name='onvg') for df in idfs_run_2]
onvg_run_3 = [get_last_value_iter(df, col_name='onvg') for df in idfs_run_3]
onvg_per_run = [onvg_run_1, onvg_run_2, onvg_run_3]

### Run 1

In [ ]:
scatter(hypervolume_run_1, mode='lines+markers')

In [ ]:
scatter(onvg_run_1, mode='lines+markers')

### Run 2

In [ ]:
scatter(hypervolume_run_2, mode='lines+markers')

In [ ]:
scatter(onvg_run_2, mode='lines+markers')

### Run 3

In [ ]:
scatter(hypervolume_run_3, mode='lines+markers')

In [ ]:
scatter(onvg_run_3, mode='lines+markers')

## Performance Indicators: Average and Standard Deviation across the 3 runs

### Layout

In [ ]:
# Default layout for the pareto fronts graphs
layout_indicators = lambda x: go.Layout(
        template="plotly_white",
        autosize=False,
        # Define plot size
        width=850,
        height=500,
        # Legend Position
        legend=dict(
            orientation='h',
            x=0.02,
            y=-0.25
        ),
        # Define axis
        xaxis=dict(
            title="Number of Evaluations",
            range=[1, 630],
            # autorange=True,
            showgrid=True,
            zeroline=True,
            showline=True,
            ticks='',
            showticklabels=True,
            tickvals = np.array(range(pop_size, 600, pop_size)),
        ),
        yaxis=dict(
            title=x,
            showgrid=True,
            zeroline=False,
            showline=True,
            ticks='',
            showticklabels=True
        )
)

### Hypervolume Indicator

In [ ]:
hypervolume_avg, hypervolume_std = indicators_results(hypervolume_per_run)

In [ ]:
fig=scatter(hypervolume_avg, mode='lines+markers',
        layout=layout_indicators('Hypervolume'),
        colorscale=None,
        colors=['#ffb300', '#FF8C61', #PSOs colors
                '#994b45', '#d96483', '#c78ba1', '#5c374c', # EAs colors
                '#11706d', '#42c7c3', '#a0e0e8'], # MBs colors
       )

fig

In [ ]:
scatter(hypervolume_std, mode='lines+markers', layout=layout_indicators('Hypervolume'))

### ONVG Indicator

In [ ]:
onvg_avg, onvg_std = indicators_results(onvg_per_run)

In [ ]:
scatter(onvg_avg, mode='lines+markers',
        layout=layout_indicators('ONVG'),
        colorscale=None,
         colors=['#ffb300', '#FF8C61', #PSOs colors
                '#994b45', '#d96483', '#c78ba1', '#5c374c', # EAs colors
                '#11706d', '#42c7c3', '#a0e0e8'], # MBs colors
       )

In [ ]:
scatter(onvg_std, mode='lines+markers', layout=layout_indicators('ONVG'))

__References__

\[1\] Russo, L. M. S., & Francisco, A. P. (2014). _Extending quick hypervolume_. Journal of Heuristics, 22(3), 245–271.

\[2\] Veldhuizen, D. V. (1999). _Multi Objective evolutionary algorithms: Classifications, Analysis, New Innovations. Multi Objective evolutionary algorithms_. Air Force Institute of Technology, Wright Patterson, Ohio.

\[3\] Schott, J. R. (1995). _Fault Tolerant Design Using Single and Multicriteria Genetic Algorithm Optimization_. Massachusetts Institute of Technology, Boston, MA.